#Install and load packages

In [0]:
!pip install azure.cognitiveservices.vision.computervision
!pip install PIL
!pip install Pillow


In [0]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import TextRecognitionMode
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import PIL
from PIL import Image
import os
import sys
import time
import pandas as pd

In [0]:
subscription_key = '10f93b9fbccd48209981e11e3c672401'
endpoint = 'https://compvisioncentralus.cognitiveservices.azure.com/'
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
remote_image_url = "https://raw.githubusercontent.com/sahilsaxena21/Microsoft_case_competition/master/img_1_jpeg.jpg"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

It is assumed that the security surveillance system within each store is able record pictures and snapshots on a regular schedule. It is to be noted that in this example, the image is saved in a remote link. For a more feasible implementation, our team will have to look into building a pipeline to continuously feed the images recorded by the security surveillance system to Azure Blob storage.

#Get Image Description

In [0]:
'''
Describe an Image - remote
This example describes the contents of an image with the confidence score.
'''
print("===== Describe an image - remote =====")
# Call API
description_results = computervision_client.describe_image(remote_image_url )

# Get the captions (descriptions) from the response, with confidence level
print("Description of remote image: ")
if (len(description_results.captions) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

===== Describe an image - remote =====
Description of remote image: 
'a group of people standing in front of a shop' with confidence 70.87%

#Get Faces

In [0]:
print("===== Detect Faces - remote =====")
# Get an image with faces
remote_image_url_faces = "https://raw.githubusercontent.com/sahilsaxena21/Microsoft_case_competition/master/img_1_jpeg.jpg"
# Select the visual feature(s) you want.
remote_image_features = ["faces"]
# Call the API with remote URL and features
detect_faces_results_remote = computervision_client.analyze_image(remote_image_url_faces, remote_image_features)

# Print the results with gender, age, and bounding box
print("Faces in the remote image: ")
if (len(detect_faces_results_remote.faces) == 0):
    print("No faces detected.")
else:
    for face in detect_faces_results_remote.faces:
        print("'{}' of age {} at location {}, {}, {}, {}".format(face.gender, face.age, \
        face.face_rectangle.left, face.face_rectangle.top, \
        face.face_rectangle.left + face.face_rectangle.width, \
        face.face_rectangle.top + face.face_rectangle.height))
# </snippet_faces>
print()

===== Detect Faces - remote =====
Faces in the remote image: 
No faces detected.

We see that no faces are detected from the example image. Here, we wanted to acknowledge this possibility. In such cases, the gender and age data will be marked as N/A. For the purposes of illustration of the metadata generated from the image analysis, we fill in these values with synthetic data in the code block below.

#Get location of people

We use the bounding box cordinates for each person to record the location of the person within the store. An example metadata generated from the image analytics is illustrated.

In [0]:
#use the bounding box of each person to get the count of people and their location within the store
'''
Detect Objects - remote
This example detects different kinds of objects with bounding boxes in a remote image.
'''

person_count = 0
person_loc_list = []
person_gender_list = []
person_age_list = []

df = pd.DataFrame(columns = ["Timestamp", "Age", "Gender", "location_category", "bestbuy_employee"])

print("===== Detect Objects - remote =====")
# Get URL image with different objects
remote_image_url_objects = "https://raw.githubusercontent.com/sahilsaxena21/Micorosoft_case_competition/master/img_1_jpeg.jpg"
# Call API with URL
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url_objects)

# Print detected objects results with bounding boxes
print("Detecting objects in remote image:")
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
      #if the object is a person, get the bounding box location
      if object.object_property == "person":
        person_count = person_count + 1
        if (object.rectangle.x < 55) and (object.rectangle.y > 80) and (object.rectangle.y < 120) :
          person_location = "cashier"
          person_loc_list.append("cashier")

          to_append = ["04/03/2020 19:53:25", 52, "Female", person_location, "No"]
          df_length = len(df)
          df.loc[df_length] = to_append
          
        elif object.rectangle.x > 590:
          person_location = "entrance"
          person_loc_list.append("entrance")

          to_append = ["04/03/2020 19:53:25", 32, "Female", person_location, "No"]
          df_length = len(df)
          df.loc[df_length] = to_append
          
        elif (object.rectangle.x > 190) and (object.rectangle.x < 210):
          person_location = "Isle_1"
          person_loc_list.append("Isle_1")

          to_append = ["04/03/2020 19:53:25", 22, "Female", person_location, "No"]
          df_length = len(df)
          df.loc[df_length] = to_append
          
        elif (object.rectangle.x > 440) and (object.rectangle.x < 460):
          person_location = "Isle_2"
          person_loc_list.append("Isle_2")

          to_append = ["04/03/2020 19:53:25", 41, "Male", person_location, "No"]
          df_length = len(df)
          df.loc[df_length] = to_append
      
      
                    
print("Total number of people in image = ", person_count)
print(df)      
          
          

===== Detect Objects - remote =====
Detecting objects in remote image:
Total number of people in image = 7
 Timestamp Age Gender location_category bestbuy_employee
0 04/03/2020 19:53:25 41 Male Isle_2 No
1 04/03/2020 19:53:25 52 Female cashier No
2 04/03/2020 19:53:25 22 Female Isle_1 No
3 04/03/2020 19:53:25 22 Female Isle_1 No
4 04/03/2020 19:53:25 32 Female entrance No
5 04/03/2020 19:53:25 32 Female entrance No
6 04/03/2020 19:53:25 32 Female entrance No